In [1]:
from openai import OpenAI

In [2]:
client = OpenAI()

In [199]:
import wikipediaapi
import requests
from bs4 import BeautifulSoup as bs
from langchain.utilities import WikipediaAPIWrapper
from langchain.tools import WikipediaQueryRun
from langchain_community.document_loaders import WebBaseLoader
import json



def QueryResearchUrl(input):
    query = input['query']
    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    url = list(wikipedia.run(query))
    return wikipedia.run(query)

# def QueryResearchUrl(input):
#     query = input['query']
#     wiki = wikipediaapi.Wikipedia(
#             user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"
#         )
#     page = wiki.page(query)
#     if page.exists():
#         return page.fullurl
#     else:
#         return None

def UrlContentScrappingTool(input):
    url = input['url']
    loader = WebBaseLoader(url)
    docs = loader.load()
    return docs[0].page_content.replace('\n','')

# def UrlContentScrappingTool(input):
#     url = input['url']    
#     res = requests.get(url)
#     soup = bs(res.text, 'html.parser')
#     paragraphs = soup.find_all('p')
#     content = "\n".join([para.text for para in paragraphs])

#     co = json.dumps(content)
#     return co


def SaveToFileTool(input):
    content = input['content']
    filename='research2.txt'
    with open(f'./files/{filename}', 'w') as f:
        f.write(content)
    return f"{filename}에 저장 완료"


functions = [
    {
        'type': 'function',
        'function': {
            'name': 'QueryResearchUrl',
            'description': 'Tool to find Wikipedia URL for a query',
            'parameters': {
                'type': 'object',
                'properties': {
                    'query': {
                        'type': 'string',
                        'description': 'What comes in when you invoke the agent'
                    }
                },
                'required': ['query']
            }
        }
    },
    {
        'type': 'function',
        'function': {
            'name': 'UrlContentScrappingTool',
            'description': 'A tool that accesses URLs and scrapes content',
            'parameters': {
                'type': 'object',
                'properties': {
                    'url': {
                        'type': 'string',
                        'description': 'URL from QueryResearchUrlTool'
                    }
                },
                'required': ['url']
            }
        }
    },
    {
        'type': 'function',
        'function': {
            'name': 'SaveToFileTool',
            'description': 'Save the content returned with UrlContentScrappingTool as a file.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'content': {
                        'type': 'string',
                        'description': 'Content to be saved to the text file'
                    }
                },
                'required': ['content']
            }
        }
    }
]


# functions = [
#     {
#         'type': 'function',
#         'function': {
#             'name': 'QueryResearchUrl',
#             'description': 'Tool to find Wikipedia URL for a query',
#             'parameters': {
#                 'type': 'object',
#                 'properties': {
#                     'query': {
#                         'type': 'string',
#                         'description': 'What comes in when you invoke the agent'
#                     }
#                 },
#                 'required': ['query']
#             }
#         }
#     },
#     {
#         'type': 'function',
#         'function': {
#             'name': 'UrlContentScrappingTool',
#             'description': 'A tool that accesses URLs and scrapes content',
#             'parameters': {
#                 'type': 'object',
#                 'properties': {
#                     'url': {
#                         'type': 'string',
#                         'description': 'URL from QueryResearchUrlTool'
#                     }
#                 },
#                 'required': ['url']
#             }
#         }
#     },
#     {
#         'type': 'function',
#         'function': {
#             'name': 'SaveToFileTool',
#             'description': 'Save the content returned with UrlContentScrappingTool as a file.',
#             'parameters': {
#                 'type': 'object',
#                 'properties': {
#                     'content': {
#                         'type': 'string',
#                         'description': 'Content to be saved to the text file'
#                     }
#                 },
#                 'required': ['content']
#             }
#         }
#     }
# ]

In [200]:
assistant = client.beta.assistants.create(
    name="research assistant",
    instructions="You are a capable assistant who helps me find what I want. When you receive a query, find the URL in Wikipedia and scrape the content. After scraping, save it as a text file.",
    tools=functions,
    model="gpt-4o-mini"
)
print(assistant)

Assistant(id='asst_clLVGLrllJaf3OzjDtNb6wZU', created_at=1726325066, description=None, instructions='You are a capable assistant who helps me find what I want. When you receive a query, find the URL in Wikipedia and scrape the content. After scraping, save it as a text file.', metadata={}, model='gpt-4o-mini', name='research assistant', object='assistant', tools=[FunctionTool(function=FunctionDefinition(name='QueryResearchUrl', description='Tool to find Wikipedia URL for a query', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'What comes in when you invoke the agent'}}, 'required': ['query']}, strict=False), type='function'), FunctionTool(function=FunctionDefinition(name='UrlContentScrappingTool', description='A tool that accesses URLs and scrapes content', parameters={'type': 'object', 'properties': {'url': {'type': 'string', 'description': 'URL from QueryResearchUrlTool'}}, 'required': ['url']}, strict=False), type='function'), FunctionTool(f

In [251]:
thread = client.beta.threads.create(
    messages=[
        {
            'role': 'user',
            'content': 'Research about the XZ backdoor.'
        }
    ]
)
print(thread)

Thread(id='thread_WbDbs64gms09mIiK5j7mj9Kz', created_at=1726327104, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [252]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)
print(list(messages))

[Message(id='msg_lBnyyvy0Ja3GIGFZgeiedFp9', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Research about the XZ backdoor.'), type='text')], created_at=1726327104, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_WbDbs64gms09mIiK5j7mj9Kz')]


In [253]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)
print(run)

Run(id='run_2LIMfpo7iQjdV3RfbExqeQlj', assistant_id='asst_clLVGLrllJaf3OzjDtNb6wZU', cancelled_at=None, completed_at=None, created_at=1726327107, expires_at=1726327707, failed_at=None, incomplete_details=None, instructions='You are a capable assistant who helps me find what I want. When you receive a query, find the URL in Wikipedia and scrape the content. After scraping, save it as a text file.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_WbDbs64gms09mIiK5j7mj9Kz', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='QueryResearchUrl', description='Tool to find Wikipedia URL for a query', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'What comes in when you invoke the agent'}}, 'required': ['query']}, strict=False), type=

In [254]:
run_retrieve = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print(run_retrieve)

Run(id='run_2LIMfpo7iQjdV3RfbExqeQlj', assistant_id='asst_clLVGLrllJaf3OzjDtNb6wZU', cancelled_at=None, completed_at=None, created_at=1726327107, expires_at=1726327707, failed_at=None, incomplete_details=None, instructions='You are a capable assistant who helps me find what I want. When you receive a query, find the URL in Wikipedia and scrape the content. After scraping, save it as a text file.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_75NmsIUtId4gC0LpSbMbR8Sl', function=Function(arguments='{"query":"XZ backdoor"}', name='QueryResearchUrl'), type='function')]), type='submit_tool_outputs'), response_format='auto', started_at=1726327108, status='requires_action', thread_id='thread_WbDbs64gms09mIiK5j7mj9Kz', tool_choice='auto', tools=[FunctionTool

In [256]:
new_run = client.beta.threads.runs.submit_tool_outputs(
    run_id=run.id,
    thread_id=thread.id,
    tool_outputs=[
        {
            'output': QueryResearchUrl({"query":"XZ backdoor"}),
            'tool_call_id': 'call_75NmsIUtId4gC0LpSbMbR8Sl'
        }
    ]
)
print(new_run)

Run(id='run_2LIMfpo7iQjdV3RfbExqeQlj', assistant_id='asst_clLVGLrllJaf3OzjDtNb6wZU', cancelled_at=None, completed_at=None, created_at=1726327107, expires_at=1726327707, failed_at=None, incomplete_details=None, instructions='You are a capable assistant who helps me find what I want. When you receive a query, find the URL in Wikipedia and scrape the content. After scraping, save it as a text file.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1726327108, status='queued', thread_id='thread_WbDbs64gms09mIiK5j7mj9Kz', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='QueryResearchUrl', description='Tool to find Wikipedia URL for a query', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'What comes in when you invoke the agent'}}, 'required': ['query']}, strict=False),

In [257]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)
print(list(messages))

[Message(id='msg_lBnyyvy0Ja3GIGFZgeiedFp9', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Research about the XZ backdoor.'), type='text')], created_at=1726327104, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_WbDbs64gms09mIiK5j7mj9Kz')]


In [258]:
run_retrieve = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print(run_retrieve)

Run(id='run_2LIMfpo7iQjdV3RfbExqeQlj', assistant_id='asst_clLVGLrllJaf3OzjDtNb6wZU', cancelled_at=None, completed_at=None, created_at=1726327107, expires_at=1726327707, failed_at=None, incomplete_details=None, instructions='You are a capable assistant who helps me find what I want. When you receive a query, find the URL in Wikipedia and scrape the content. After scraping, save it as a text file.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_OEkH7OKq4EumfqjIfexJHnqk', function=Function(arguments='{"url": "https://en.wikipedia.org/wiki/XZ_Utils_backdoor"}', name='UrlContentScrappingTool'), type='function'), RequiredActionFunctionToolCall(id='call_5fyQTHOqibIZeemxjrgs8ihO', function=Function(arguments='{"url": "https://en.wikipedia.org/wiki/XZ_Utils"}'

In [259]:
required_actions = run_retrieve.required_action.submit_tool_outputs.tool_calls
new_run = client.beta.threads.runs.submit_tool_outputs(
    run_id=run.id,
    thread_id=thread.id,
    tool_outputs=[
    {
        'output': UrlContentScrappingTool(
            {
                "url": json.loads(action.function.arguments)['url']
            }),
        'tool_call_id': action.id

    } for action in required_actions
]
)
print(new_run)

Run(id='run_2LIMfpo7iQjdV3RfbExqeQlj', assistant_id='asst_clLVGLrllJaf3OzjDtNb6wZU', cancelled_at=None, completed_at=None, created_at=1726327107, expires_at=1726327707, failed_at=None, incomplete_details=None, instructions='You are a capable assistant who helps me find what I want. When you receive a query, find the URL in Wikipedia and scrape the content. After scraping, save it as a text file.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1726327147, status='queued', thread_id='thread_WbDbs64gms09mIiK5j7mj9Kz', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='QueryResearchUrl', description='Tool to find Wikipedia URL for a query', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'What comes in when you invoke the agent'}}, 'required': ['query']}, strict=False),

In [260]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)
print(list(messages))

[Message(id='msg_lBnyyvy0Ja3GIGFZgeiedFp9', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Research about the XZ backdoor.'), type='text')], created_at=1726327104, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_WbDbs64gms09mIiK5j7mj9Kz')]


In [262]:
run_retrieve = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print(run_retrieve)

Run(id='run_2LIMfpo7iQjdV3RfbExqeQlj', assistant_id='asst_clLVGLrllJaf3OzjDtNb6wZU', cancelled_at=None, completed_at=None, created_at=1726327107, expires_at=1726327707, failed_at=None, incomplete_details=None, instructions='You are a capable assistant who helps me find what I want. When you receive a query, find the URL in Wikipedia and scrape the content. After scraping, save it as a text file.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_2VLeoavaK7Qci1lhlCXxbM7l', function=Function(arguments='{"content":"# XZ Backdoor\\n\\n## Overview\\nIn February 2024, a malicious backdoor was introduced to the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name \\"Jia Tan\\". The backdoor allows an attacker poss

In [263]:
new_run = client.beta.threads.runs.submit_tool_outputs(
    run_id=run.id,
    thread_id=thread.id,
    tool_outputs=[
        {
            'output': SaveToFileTool({"content":"""# XZ Backdoor\\n\\n## Overview\\nIn February 2024, a malicious backdoor was introduced to the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 by an account using the name \\"Jia Tan\\". The backdoor allows an attacker possessing a specific Ed448 private key to execute remote code on the affected Linux system. This issue has been assigned the CVE identifier CVE-2024-3094 and has been given a CVSS score of 10.0, the highest possible score.\\n\\nAlthough xz is a commonly installed utility in most Linux distributions, the backdoored version had not been widely deployed to production systems at the time of its discovery, but was found in development versions of major distributions. Software developer Andres Freund made the announcement of the backdoor on March 29, 2024.\\n\\n## Background\\nAndres Freund discovered the backdoor after investigating a performance regression in Debian Sid, where he observed high CPU usage and errors in memory debugging tools like Valgrind during SSH connections. His findings were reported to the Openwall Project\'s open-source security mailing list, drawing attention from various software vendors.\\n\\nThe attacker, presumed to be Jia Tan, manipulated the project to gain the trust needed to introduce the backdoor. Through a series of sockpuppet accounts, they gradually attained a position of co-maintainer within the XZ Utils project and signed off on the compromised versions of the software.\\n\\n## Mechanism\\nThe malicious code exists in the 5.6.0 and 5.6.1 releases of the XZ Utils software package. The exploit remains dormant unless a specific third-party patch of the SSH server is applied. Under the right conditions, it could allow a malicious actor to bypass authentication and gain remote access to the entire system.\\n\\nThe backdoor mechanism includes two compressed test files that separate the malicious binary code, which requires extraction and injection into the program. By modifying existing functions in OpenSSH through the shared library loading mechanism, the backdoor gains administrator access when executed properly on a compromised system.\\n\\n## Response\\n### Remediation\\nThe U.S. federal Cybersecurity and Infrastructure Security Agency (CISA) issued a security advisory, advising affected devices to revert to a previous version of the software. Prominent Linux vendors have rolled back their versions to safe states, including Red Hat, SUSE, and Debian. Canonical postponed the release of Ubuntu 24.04 LTS and initiated a total binary rebuild of all packages to prevent possible exploitation.\\n\\n### Broader Response\\nThe incident raised concerns about the risks associated with open-source software relying on unpaid volunteers, highlighting the potential for critical utilities to be compromised in unnoticed and damaging ways. Security expert Alex Stamos suggested that had the backdoor gone undetected, it could have led to widespread unauthorized access to sensitive SSH implementations worldwide, posing an enormous security threat.\\n\\n# Simultaneously Relevant Terms\\n## XZ Utils\\nXZ Utils is a free software command-line tool for lossless data compression, popular in Unix-like OS and later on Windows. It utilizes the Lempel-Ziv-Markov chain algorithm for compression and decompression. XZ\'s ability to achieve high compression rates makes it widely used in various Linux distributions for compressing software packages.\\n\\n## Backdoor (Computing)\\nA backdoor refers to a method for bypassing normal security measures, providing unauthorized access to a system. Backdoors can reside in software, hardware, or even firmware and are often used for sustaining covert access to digital environments. They can be deliberately embedded by developers or maliciously inserted by attackers, presenting major security risks."""}),
            'tool_call_id': 'call_2VLeoavaK7Qci1lhlCXxbM7l'
        }
    ]
)
print(new_run)

Run(id='run_2LIMfpo7iQjdV3RfbExqeQlj', assistant_id='asst_clLVGLrllJaf3OzjDtNb6wZU', cancelled_at=None, completed_at=None, created_at=1726327107, expires_at=1726327707, failed_at=None, incomplete_details=None, instructions='You are a capable assistant who helps me find what I want. When you receive a query, find the URL in Wikipedia and scrape the content. After scraping, save it as a text file.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1726327167, status='queued', thread_id='thread_WbDbs64gms09mIiK5j7mj9Kz', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='QueryResearchUrl', description='Tool to find Wikipedia URL for a query', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'What comes in when you invoke the agent'}}, 'required': ['query']}, strict=False),

In [264]:
run_retrieve = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)
print(run_retrieve)

Run(id='run_2LIMfpo7iQjdV3RfbExqeQlj', assistant_id='asst_clLVGLrllJaf3OzjDtNb6wZU', cancelled_at=None, completed_at=1726327260, created_at=1726327107, expires_at=None, failed_at=None, incomplete_details=None, instructions='You are a capable assistant who helps me find what I want. When you receive a query, find the URL in Wikipedia and scrape the content. After scraping, save it as a text file.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o-mini', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1726327259, status='completed', thread_id='thread_WbDbs64gms09mIiK5j7mj9Kz', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='QueryResearchUrl', description='Tool to find Wikipedia URL for a query', parameters={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'What comes in when you invoke the agent'}}, 'required': ['query']}, strict=Fals

In [265]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)
print(list(messages))

[Message(id='msg_imKMyQbzSmgdabilLM1WEx5v', assistant_id='asst_clLVGLrllJaf3OzjDtNb6wZU', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='The research about the XZ backdoor has been compiled into a text file. You can access and download it through the link below:\n\n[Download XZ Backdoor Research](sandbox:/research2.txt)'), type='text')], created_at=1726327260, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_2LIMfpo7iQjdV3RfbExqeQlj', status=None, thread_id='thread_WbDbs64gms09mIiK5j7mj9Kz'), Message(id='msg_lBnyyvy0Ja3GIGFZgeiedFp9', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Research about the XZ backdoor.'), type='text')], created_at=1726327104, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_WbDbs64gms09mIiK5j7mj9